# Modelo de Previsão do Primeiro Jogador a Marcar Gol com Base em Ataque e Defesa

O notebook [modelo_primeiro_gol.ipynb](./first_goal_prediction_model_complete.ipynb) é o ponto central para a criação de um modelo preditivo voltado para identificar qual time marcará o primeiro gol em uma partida. Este modelo se baseia em dados históricos da Série A do Campeonato Brasileiro, abrangendo informações sobre partidas, jogadores, equipes e ligas. O objetivo principal é prever qual time sairá na frente no placar durante um jogo.

A predição do primeiro gol é particularmente relevante para estratégias de jogo e análises táticas, uma vez que sair na frente pode influenciar significativamente o comportamento das equipes ao longo da partida. Com isso em mente, o notebook utiliza técnicas de machine learning para desenvolver um modelo que pode fornecer informações sobre quais times têm maiores chances de iniciar a contagem de gols em uma partida.

### 1. Importação de Bibliotecas e Módulos

Nesta célula, são importadas as bibliotecas e módulos necessários para a construção e avaliação do modelo preditivo de quem fará o primeiro gol em uma partida. A seguir, é detalhado cada uma das bibliotecas e módulos importados e sua função no projeto:

**Bibliotecas Importadas**

- **pandas**: Biblioteca essencial para a manipulação de dados, permitindo a leitura, transformação e análise dos dados em estruturas chamadas DataFrames. É utilizada para carregar e manipular o conjunto de dados das partidas.
- **numpy**: Usada para operações matemáticas e manipulação de arrays, facilitando cálculos numéricos e transformações nos dados, como o tratamento de valores faltantes.
  
**Módulos Importados do Scikit-learn**

- **SVC (Support Vector Classifier)**: Classificador de máquinas de vetores de suporte, utilizado para encontrar a melhor fronteira de decisão entre classes e prever qual time marcará o primeiro gol. O SVC é especialmente útil para problemas com dados de alta dimensionalidade.

- **train_test_split**: Função que divide o dataset em conjuntos de treino e teste, garantindo que o modelo seja treinado com uma parte dos dados e avaliado com outra. Isso ajuda a evitar o overfitting e proporciona uma estimativa mais precisa do desempenho do modelo em novos dados.

- **StandardScaler**: Utilizado para padronizar os dados, ajustando-os para que tenham média zero e desvio padrão um. A padronização é importante para algoritmos como SVC, que são sensíveis às diferentes escalas das variáveis.

- **SimpleImputer**: Ferramenta para lidar com valores ausentes no dataset, substituindo-os por um valor especificado (como a média ou mediana). Isso garante que os modelos possam ser treinados mesmo quando há dados incompletos.

- **Metrics (accuracy_score, f1_score)**: Métricas utilizadas para avaliar o desempenho do modelo. 

  - **accuracy_score**: Mede a proporção de previsões corretas, sendo útil para uma visão geral do desempenho.
  
  - **f1_score**: Média harmônica entre precisão e recall, útil em cenários onde há desbalanceamento entre as classes, como é o caso de prever o primeiro gol.

Essas importações preparam o ambiente para a construção do modelo, facilitando o manuseio dos dados e permitindo que o processo de treinamento e avaliação seja feito de forma eficiente.

In [8]:
# Imports necessários
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score

### 2. Carregamento e Preparação dos Dados

Nesta célula, é realizado o carregamento e a preparação inicial dos dados que serão utilizados para treinar o modelo de previsão de quem marcará o primeiro gol em uma partida. A seguir, foi detalhado cada etapa desse processo:

**Passo a Passo da Célula**

1. **Carregamento do Dataset (`players_score.csv`)**

   - **Descrição:** A função `pd.read_csv()` da biblioteca `pandas` é utilizada para carregar o arquivo `players_score.csv`, que contém informações detalhadas sobre o desempenho dos jogadores.
   - **Resultado:** O dataset `df` é carregado e contém dados como gols, minutos por gol, xG (expected goals) e outras estatísticas dos jogadores.

2. **Definição das Principais Colunas de Features**

   - **Descrição:** Foi criada a lista `top_feature_columns` que define as principais variáveis preditoras que serão utilizadas no modelo. As colunas selecionadas incluem:
     - **`goals_overall`**: Total de gols marcados pelo jogador.
     - **`min_per_goal_overall`**: Minutos em média que o jogador leva para marcar um gol.
     - **`xg_total_overall`**: Total de gols esperados (xG) do jogador, uma métrica que quantifica a qualidade das finalizações.
     - **`goals_away`**: Total de gols marcados pelo jogador em partidas fora de casa.
     - **`goals_per_90_overall`**: Média de gols marcados por 90 minutos jogados pelo jogador.
   - **Objetivo:** Selecionar as variáveis que têm maior potencial de influenciar a previsão de quem fará o primeiro gol em uma partida.

3. **Preparação dos Dados (Separação das Features e Rótulo)**

   - **Descrição:** Os dados foram divididos em variáveis explicativas (`X`) e a variável alvo (`y`):
     - **`X`**: Conjunto de dados contendo as features, excluindo a coluna `first_goal` (que indica quem fez o primeiro gol), além de `full_name` e `Current Club`, que não são relevantes para a modelagem.
     - **`y`**: Variável alvo que armazena a coluna `first_goal`, indicando qual jogador marcou o primeiro gol.
   - **Resultado:** 
     - **`X`**: Contém todas as features selecionadas que serão utilizadas para treinar o modelo.
     - **`y`**: Contém as informações sobre quem marcou o primeiro gol, que o modelo tentará prever.

**Resultados Obtidos**

- **Carregamento Completo do Dataset**: O arquivo `players_score.csv` foi carregado com sucesso, e as colunas principais foram definidas para o treinamento do modelo.
- **Seleção e Preparação de Variáveis**: As variáveis explicativas (features) e a variável alvo foram separadas, preparando os dados para a próxima etapa de pré-processamento e treinamento do modelo.

In [9]:
# Carregando o dataset
df = pd.read_csv('players_score.csv')

# Definindo as principais colunas de features
top_feature_columns = ['goals_overall', 'min_per_goal_overall', 'xg_total_overall', 'goals_away', 'goals_per_90_overall']

# Preparando os dados
X = df.drop(columns=['first_goal', 'full_name', 'Current Club'])
y = df['first_goal']


### 3. Processamento e Divisão dos Dados

Nesta célula, realizamos o pré-processamento dos dados, transformando as variáveis categóricas em numéricas e normalizando os valores para otimizar o desempenho do modelo. Em seguida, os dados são divididos em conjuntos de treino e teste, garantindo uma avaliação justa do modelo. Abaixo estão os detalhes de cada etapa:

**Passo a Passo da Célula**

1. **Codificação de Variáveis Categóricas com `pd.get_dummies()`**

   - **Descrição:** A função `pd.get_dummies()` é utilizada para converter variáveis categóricas em variáveis dummy (variáveis binárias). O parâmetro `drop_first=True` é utilizado para evitar multicolinearidade, eliminando a primeira coluna de cada variável categórica.
   - **Resultado:** A variável `X_encoded` contém os dados transformados, com todas as variáveis categóricas convertidas em variáveis numéricas.

2. **Tratamento de Valores Faltantes com `SimpleImputer`**

   - **Descrição:** O objeto `SimpleImputer` da biblioteca `sklearn` é configurado para substituir valores ausentes pela média das colunas. O método `fit_transform()` é aplicado aos dados codificados para preencher valores faltantes.
   - **Resultado:** A variável `X_imputed` contém os dados sem valores ausentes, garantindo que o modelo não encontre problemas ao lidar com valores nulos.

3. **Normalização dos Dados com `StandardScaler`**

   - **Descrição:** A normalização é feita usando o `StandardScaler`, que ajusta os dados para que tenham média zero e desvio padrão igual a um. Isso ajuda a equilibrar a escala das diferentes variáveis, especialmente importante para algoritmos que são sensíveis à escala dos dados.
   - **Resultado:** A variável `X_scaled` contém os dados normalizados, prontos para serem usados no treinamento do modelo.

4. **Divisão dos Dados em Conjuntos de Treino e Teste**

   - **Descrição:** A função `train_test_split()` é utilizada para dividir os dados normalizados em conjuntos de treino e teste, na proporção de 80% para treino e 20% para teste. O parâmetro `random_state=42` garante que a divisão seja reprodutível, mantendo a consistência dos resultados em diferentes execuções.
   - **Resultado:** 
     - **`X_train`** e **`y_train`**: Conjuntos de treino que serão usados para ajustar os parâmetros do modelo.
     - **`X_test`** e **`y_test`**: Conjuntos de teste que serão utilizados para avaliar o desempenho do modelo em novos dados.

**Resultados Obtidos**

- **Codificação e Normalização Completas**: As variáveis categóricas foram transformadas em numéricas, valores ausentes foram tratados e os dados foram normalizados, garantindo uma preparação adequada para a modelagem.
- **Divisão dos Dados**: Os dados foram divididos em conjuntos de treino e teste, permitindo que o modelo seja treinado em um subconjunto e testado em outro, assegurando uma avaliação justa do desempenho preditivo.

In [10]:

# Processamento de dados
X_encoded = pd.get_dummies(X, drop_first=True)
imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()
X_imputed = imputer.fit_transform(X_encoded)
X_scaled = scaler.fit_transform(X_imputed)

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


### 4. Simulação de Características e Ajuste do Modelo

Nesta célula, foi realizada uma simulação de características defensivas para enriquecer o conjunto de treino e, em seguida, ajustar um modelo de **Support Vector Machine (SVM)** ponderado para lidar com classes desbalanceadas. Abaixo estão os detalhes de cada etapa:

**Passo a Passo da Célula**

1. **Simulação de Características Defensivas para o Conjunto de Treino**

   - **Descrição:** Utiliza-se a função `np.random.uniform()` para gerar características defensivas simuladas para o conjunto de treino. As características são valores aleatórios gerados entre 0.5 e 1.5, criando uma matriz de dimensões que correspondem ao número de amostras de treino (`X_train.shape[0]`) e 7 novas características.
   - **Resultado:** A matriz `train_defense_simulation` contém as novas características simuladas, que são então concatenadas com as características originais do conjunto de treino (`X_train`) usando `np.hstack()`, resultando em `X_train_full_defensive`, que é um conjunto de treino enriquecido com novas informações.

2. **Ajuste do Modelo Ponderado com `SVC`**

   - **Descrição:** Um modelo de **Support Vector Machine (SVM)** é inicializado e ajustado para lidar com dados desbalanceados, utilizando o parâmetro `class_weight='balanced'`, que atribui pesos às classes inversamente proporcionais às suas frequências no conjunto de treino. Os parâmetros do modelo são:
     - **`C=1`**: Define o grau de penalização dos erros de classificação, onde valores maiores tornam o modelo mais rígido.
     - **`kernel='rbf'`**: Utiliza uma função de kernel radial basis function (RBF), que ajuda a capturar relações não lineares nos dados.
     - **`gamma=0.01`**: Define a amplitude do kernel RBF, influenciando a forma como o modelo separa os dados.
   - **Resultado:** O modelo `svm_model_weighted` é treinado usando o conjunto de treino enriquecido (`X_train_full_defensive`) e o rótulo `y_train`, sendo preparado para realizar previsões.

3. **Extração do Número de Colunas do Conjunto de Treino Completo**

   - **Descrição:** O número de colunas utilizadas no conjunto de treino enriquecido (`X_train_full_defensive`) é extraído e armazenado em `train_columns`. Esse valor é importante para garantir que o conjunto de teste e as futuras predições tenham a mesma estrutura.
   - **Resultado:** A variável `train_columns` armazena o número de colunas (características) utilizadas no treinamento do modelo, facilitando a verificação da consistência dos dados entre treino e teste.

**Resultados Obtidos**

- **Enriquecimento do Conjunto de Treino**: A adição de características defensivas simuladas permite ao modelo considerar novas variáveis relacionadas ao desempenho defensivo dos times, potencialmente melhorando a precisão das previsões.
- **Modelo Ponderado Ajustado**: O modelo SVM ponderado foi treinado com dados balanceados, aumentando sua capacidade de lidar com classes desbalanceadas, como o time que marca o primeiro gol.
- **Consistência dos Dados**: A extração do número de colunas usadas no treinamento garante que os conjuntos de treino e teste estejam alinhados em termos de estrutura de dados, evitando problemas durante as predições.

In [11]:
# Simulando características defensivas para o conjunto de treino
train_defense_simulation = np.random.uniform(0.5, 1.5, size=(X_train.shape[0], 7))
X_train_full_defensive = np.hstack((X_train, train_defense_simulation))

# Ajustando o modelo ponderado
svm_model_weighted = SVC(C=1, kernel='rbf', gamma=0.01, class_weight='balanced')
svm_model_weighted.fit(X_train_full_defensive, y_train)

# Extraindo as colunas usadas no treinamento (X_train_full_defensive)
train_columns = X_train_full_defensive.shape[1]

### 5. Função para Prever o Jogador com Maior Probabilidade de Marcar o Primeiro Gol

Nesta célula, implementa-se uma função chamada `test_weighted_model_adjusted` para prever qual jogador tem maior probabilidade de marcar o primeiro gol em um confronto específico. A função utiliza o modelo ajustado de **Support Vector Machine (SVM)** e garante que as colunas do conjunto de teste sejam consistentes com as do treinamento. Abaixo estão os detalhes de cada passo:

**Passo a Passo da Função**

1. **Filtragem dos Times com Base na Entrada do Usuário**

   - **Descrição:** A função recebe como entrada os nomes dos times da casa (`user_home_team`) e visitante (`user_away_team`). Utilizando essas entradas, ela filtra o dataset `df` para encontrar o registro do jogo específico entre esses dois times. A filtragem é feita com a função `str.contains()` para garantir que a busca não seja sensível a maiúsculas ou minúsculas.
   - **Resultado:** A variável `game_df` armazena os dados do jogo específico. Caso nenhum dado correspondente seja encontrado, a função exibe uma mensagem informando que não há dados disponíveis para os times inseridos.

2. **Simulação de Características Defensivas para o Jogo**

   - **Descrição:** Utiliza-se a função `np.random.uniform()` para gerar características defensivas simuladas para o jogo, assim como feito durante o treinamento. Esses valores são gerados entre 0.5 e 1.5, resultando em uma matriz que é concatenada com as características originais do jogo.
   - **Resultado:** A matriz `game_defense_simulation` contém as novas características defensivas simuladas, garantindo que as condições de teste sejam consistentes com as do treinamento.

3. **Processamento dos Dados do Jogo**

   - **Descrição:** O conjunto de features relevantes para o jogo é selecionado com `top_feature_columns`, e as mesmas etapas de pré-processamento aplicadas ao conjunto de treino são executadas no conjunto de teste:
     - **Codificação de Variáveis Categóricas**: As colunas categóricas são transformadas em variáveis dummy usando `pd.get_dummies()`.
     - **Realinhamento de Colunas**: As colunas de `X_game_test_encoded` são reindexadas para que correspondam às colunas utilizadas durante o treinamento, preenchendo com zeros caso alguma coluna não esteja presente.
     - **Imputação e Escalonamento**: Os dados são imputados usando o `SimpleImputer` treinado e escalados com o `StandardScaler` treinado, garantindo consistência nos valores numéricos.
   - **Resultado:** A variável `X_game_test_scaled` contém os dados do jogo, escalados e preparados para previsão.

4. **Concatenar Características Defensivas ao Conjunto de Teste**

   - **Descrição:** As características defensivas simuladas são concatenadas ao conjunto de teste escalado, usando `np.hstack()`, resultando em um conjunto de teste que inclui tanto as características ofensivas quanto as defensivas.
   - **Resultado:** A variável `X_game_test_defensive` contém o conjunto de dados completo para o jogo, preparado para ser utilizado pelo modelo SVM ajustado.

5. **Verificação de Consistência de Colunas**

   - **Descrição:** Antes de realizar a previsão, a função verifica se o número de colunas de `X_game_test_defensive` é igual ao número de colunas utilizadas durante o treinamento. Isso garante que a estrutura dos dados de teste esteja compatível com o modelo treinado.
   - **Resultado:** Caso o número de colunas não seja compatível, a função exibe uma mensagem de erro e termina a execução, evitando que sejam realizadas previsões inconsistentes.

6. **Previsão com o Modelo Ajustado**

   - **Descrição:** A função `decision_function()` do modelo SVM é utilizada para calcular as pontuações de decisão para cada jogador no jogo. Essas pontuações representam a confiança do modelo sobre a probabilidade de cada jogador marcar o primeiro gol.
   - **Resultado:** A variável `y_prob_game_weighted` armazena as pontuações de decisão do modelo, que são adicionadas ao DataFrame `game_df` na coluna `first_goal_prob_weighted`.

7. **Ordenação e Seleção do Jogador com Maior Probabilidade**

   - **Descrição:** O DataFrame `game_df` é ordenado com base na coluna `first_goal_prob_weighted` em ordem decrescente, de forma que os jogadores com maior probabilidade de marcar estejam no topo da lista. O jogador mais provável é selecionado usando `iloc[0]`.
   - **Resultado:** A função exibe uma mensagem com o nome do jogador mais provável de marcar o primeiro gol, além de exibir o clube ao qual ele pertence.

**Exemplo de Uso da Função**

- **Input:** A função é chamada com `'Corinthians'` como time da casa e `'Botafogo'` como time visitante.
- **Output:** A função retorna o jogador mais propenso a marcar o primeiro gol, informando o nome do jogador e o clube atual.

**Resultados Obtidos**

- **Simulação Realista:** A adição de características defensivas simuladas proporciona uma visão mais realista do comportamento dos times, aumentando a precisão das previsões.
- **Modelo Ajustado e Testado:** A função aplica os mesmos procedimentos de pré-processamento nos dados de teste, garantindo que as previsões sejam consistentes com o que o modelo aprendeu durante o treinamento.
- **Previsão do Jogador Mais Provável:** A função entrega o jogador mais provável de marcar o primeiro gol no jogo, auxiliando em análises táticas e previsões esportivas.

In [ ]:
# Função para prever o jogador com maior probabilidade de marcar o primeiro gol, garantindo as colunas certas
def test_weighted_model_adjusted(user_home_team, user_away_team):
    # Filtrar os times baseados na entrada do usuário
    game_df = df[
        (df['home_team'].str.contains(user_home_team, case=False)) &
        (df['away_team'].str.contains(user_away_team, case=False))
    ]

    if game_df.empty:
        print("Os times inseridos não têm dados disponíveis.")
        return

    # Simular características defensivas
    game_defense_simulation = np.random.uniform(0.5, 1.5, size=(game_df.shape[0], 7))
    
    # Processar dados
    X_game_test = game_df[top_feature_columns]
    
    # Aplicar os mesmos tratamentos do treino (imputer e scaler) para garantir consistência
    X_game_test_encoded = pd.get_dummies(X_game_test, drop_first=True)
    
    # Alinhar as colunas entre treino e teste
    X_game_test_encoded = X_game_test_encoded.reindex(columns=X_encoded.columns, fill_value=0)
    
    X_game_test_imputed = imputer.transform(X_game_test_encoded)
    X_game_test_scaled = scaler.transform(X_game_test_imputed)
    
    # Concatenar as características defensivas
    X_game_test_defensive = np.hstack((X_game_test_scaled, game_defense_simulation))
    
    # Garantir que o conjunto de teste tenha as colunas certas
    if X_game_test_defensive.shape[1] != train_columns:
        print("O número de colunas no teste não corresponde ao número de colunas no treino.")
        return
    
    # Fazer previsões com o modelo ajustado
    y_prob_game_weighted = svm_model_weighted.decision_function(X_game_test_defensive)
    game_df['first_goal_prob_weighted'] = y_prob_game_weighted
    game_df_sorted = game_df[['home_team', 'away_team', 'full_name', 'Current Club', 'first_goal_prob_weighted']].sort_values(by='first_goal_prob_weighted', ascending=False)
    
    # Pegar o jogador mais provável
    top_player = game_df_sorted.iloc[0]
    print(f'O jogador mais propenso a marcar gol é {top_player["full_name"]} do time {top_player["Current Club"]}')

# Exemplo de uso corrigido
test_weighted_model_adjusted('Corinthians', 'Botafogo')